In [21]:
# RVM classification for a mixture of Gaussians - Rémi Lacombe - January 2018

import numpy as np
import matplotlib.pyplot as plt


In [22]:
# Creates (or imports) dataset

# Says which dataset is chosen: 0 for Ripley's Gaussians/ 1 for Pima indians/ 2 for USPS hand-written digits
set_option = 1


# Classification on Ripley's Gaussians dataset
if (set_option == 0):

    big_ripley = np.zeros((250,3))

    # Converts the dataset from text file to Python arrays
    my_file = open('synth.tr','r')

    index = 0
    number = ''

    for line in my_file:
        for i in range(len(line)):
            if (line[i] == ' '):
                if ((len(number) > 0) | (i == len(line)-1)):
                    big_ripley[index/3][index%3] = float(number)
                    index += 1
                number = ''
            else:
                number += line[i]

    my_file.close()

    X_ripley = big_ripley[:,0]
    Y_ripley = big_ripley[:,1]
    t_ripley = big_ripley[:,2]
    t_ripley[-1] = 1
    
    indices_0 = [n for n in range(125)]
    indices_1 = [n for n in range(125,250)]

    '''
    plt.scatter(X_ripley[indices_0], Y_ripley[indices_0], c='r', linewidths = 0.1, marker = 'o')
    plt.scatter(X_ripley[indices_1], Y_ripley[indices_1], c='b', linewidths = 0.1, marker = 'v')
    plt.show()
    '''
    
    # Let us choose 100 points at random in this dataset
    
    N = 100
    
    random_indices = np.zeros(N)
    for i in range(N):
        random = np.random.randint(0,250)
        while random in random_indices:
            random = np.random.randint(0,250)
        random_indices[i] = int(random)
    
    random_indices = random_indices.astype(int)
    random_indices = np.sort(random_indices)
    
    data_points = np.array([X_ripley[random_indices], Y_ripley[random_indices]])
    data_points = np.transpose(data_points)
    
    t = t_ripley[random_indices]
    
    break_index = 0
    while random_indices[break_index] < 125:
        break_index += 1

    indices_0_r = [random_indices[n] for n in range(break_index)]
    indices_1_r = [random_indices[n] for n in range(break_index,N)]
    
    plt.scatter(X_ripley[indices_0_r], Y_ripley[indices_0_r], c='r', linewidths = 0.1, marker = 'o')
    plt.scatter(X_ripley[indices_1_r], Y_ripley[indices_1_r], c='b', linewidths = 0.1, marker = 'v')
    plt.title("Ripley's Gaussians dataset with 100 data points chosen at random")
    plt.show()
    
    
# Classification on Pima diabetes dataset
if (set_option == 1):
    
    N = 200

    data_points = np.zeros((200,7))
    t = np.zeros(200)

    # Converts the dataset from text file to Python arrays
    my_file = open('pima.tr','r')

    index = 0

    for line in my_file:
        number = ''
        for i in range(len(line)):
            if (line[i] == ' '):
                if (len(number) > 0):
                    data_points[index/7][index%7] = float(number)
                    index += 1
                number = ''
            elif (i == len(line) - 2):
                t[index/7 - 1] = float(line[i])
            else:
                number += line[i]

    my_file.close()
    
    
    
# Classification on USPS hand-written digits
if (set_option == 2):
    
    N = 7291

    data_points = np.zeros((7291,256))
    t = np.zeros(7291)

    # Converts the dataset from text file to Python arrays
    my_file = open('usps.tr','r')

    index = 0
    
    it_is_a_10 = 0

    for line in my_file:
        
        number = ''
        
        for i in range(len(line)):
            
            if (it_is_a_10 == 1):
                it_is_a_10 = 0
                
            elif (i == 0):
                if (line[i+1] == '0'):
                    t[index/256] = 10
                    it_is_a_10 = 1
                else:
                    t[index/256] = int(line[i])
                
            elif (line[i] == ':'):
                number = ''
                
            elif (line[i] == ' '):
                if (len(number) > 0):
                    if (index/256 == 7291):
                        print(line)
                    data_points[index/256][index%256] = float(number)
                    index += 1
                number = ''
                
            elif (i == len(line) - 2):
                number = ''
                j = i
                while (line[j] != ':'):
                    number += line[j]
                    j -= 1
                new_number = ''
                for k in range(len(number)):
                    new_number += number[len(number) - 1 - k]
                data_points[index/256][index%256] = float(new_number)
                index += 1
                
            else:
                number += line[i]
                
    my_file.close()
    

In [23]:
# Creates Kernel function

kernel_option = 49

def Gaussian_kernel(x,y):
    return np.exp(-sum((x-y)*(x-y))/pow(kernel_option,2))

def y(x_index, w, Phi):
    return sum(Phi[x_index]*w)

def logistic_sigmoid(y):
    return 1/(1 + np.exp(-y))


In [24]:
# Performs the classification

w = 0.01*np.ones(N+1)
w_new = w

# The hyperparameters alpha_i are initially taken to be 1
alpha = np.ones(N+1)
A = np.diag(alpha)

# Keeps track of the indices of the relevance vectors
alpha_indices = np.linspace(0,N,N+1)
alpha_indices = alpha_indices.astype(int)

# Creates the design matrix Phi
Phi = np.ones((N,N+1))

for i in range(N):
    
    for j in range(i+1):
        
        Phi[i][j+1] = Gaussian_kernel(data_points[i], data_points[j])
        Phi[j][i+1] = Phi[i][j+1]
        
new_Phi = Phi

# Deals with the special case where w_0 is not relevant
not_removed_0 = 1
        
        
# Main loop for the re-estimation of alpha
for i in range(500):
        
        
    k = len(alpha_indices)
    
    # Checks for infinite loops in the optimization algorithm
    iteration_number = 0

    update_quantity = np.ones(N+1)
    
    # Newton's optimization algorithm is used to find wmp
    while not all(abs(j) < 1e-14 for j in update_quantity):
            
        logistic_array = np.array([logistic_sigmoid(y(n, w, Phi)) for n in range(N)])
        B = np.diag(logistic_array*(1-logistic_array))

        gradient = np.dot(np.transpose(new_Phi), t - logistic_array) - np.dot(A, w_new)
        
        hessian = -np.dot(np.transpose(new_Phi), np.dot(B, new_Phi)) - A

        update_quantity = np.dot(np.linalg.inv(hessian), gradient)
    
        w_new = w_new - update_quantity
        
        w[alpha_indices] = w_new
        
        iteration_number += 1
    
        if (iteration_number == 100):
            print("OBS ! Infinite loop.")
            break
    
            
    # Updates B with the new value of wmp 
    logistic_array = np.array([logistic_sigmoid(y(n, w, Phi)) for n in range(N)])
    B = np.diag(logistic_array*(1-logistic_array))
    
    small_Sigma = np.linalg.inv(np.dot(np.transpose(new_Phi), np.dot(B, new_Phi)) + A)
    
    #w_new_test = np.dot(np.linalg.inv(A), np.dot(np.transpose(new_Phi), t - logistic_array))
    
    C = np.zeros((k, N+1))
    C[:, alpha_indices] = small_Sigma
    
    Sigma = np.zeros((N+1,N+1))
    Sigma[alpha_indices,:] = C
    
    mu = w_new
    
    gamma = 1 - alpha*np.diag(small_Sigma)
    alpha = gamma/pow(mu, 2)
    
    # Prunes basis functions for which alpha_i tends to infinity
    prune = np.where(alpha > 1e12)[0]
    
    # Keeps track of the indices that are removed
    alpha_indices = np.delete(alpha_indices, prune)
    
    if ((0 in prune) & (not_removed_0)):
        
        prune = np.delete(prune, 0)
        prune = prune - 1
        not_removed_0 = 0
        
        alpha = np.delete(alpha, 0)
        w_new = np.delete(w_new, 0)
        new_Phi = np.delete(new_Phi, 0, 1)
        
    if (len(prune) > 0):
        
        alpha = np.delete(alpha, prune)
        w_new = np.delete(w_new, prune)

        # Checks if hyperparameter of index 0 has been removed
        if (not_removed_0):
            new_Phi = np.delete(new_Phi, prune, 1)
        else:
            new_Phi = np.delete(new_Phi, prune, 1)
    
    A = np.diag(alpha)
    
    # Updates parameters w (this can also be commented to get another slightly different w)
    w = np.zeros(N+1)
    w[alpha_indices] = w_new

    
# Increase the number of iterations in the main loop if some values of w seem too small
print(alpha_indices)
print(w)


[ 44 130 147 158 187]
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -1.30207333  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.  

In [25]:
# Makes predictions and plots decision boundary

# Defines the precision of the boundary
set_size = 50

X_ripley_test = np.linspace(-1.3, 1, set_size)
Y_ripley_test = np.linspace(-0.3, 1.2, set_size)


# Computes the posterior value for every new point (OBS: the case where the bias remains is not included)
def posterior(new_point):
    
    result = 0
    
    for i in range(len(alpha_indices)):
        
        index = alpha_indices[i]
        result += w[index]*Gaussian_kernel(data_points[index-1], new_point)
        
    return(logistic_sigmoid(result))


values = np.array([[posterior((x, y)) for x in X_ripley_test] for y in Y_ripley_test])

# Computes the coordinates of the relevance vectors
a = data_points[alpha_indices-1]
X_relevance = a[:,0]
Y_relevance = a[:,1]

# Plots the decision boundary
plt.contour(X_ripley_test, Y_ripley_test, values, 0.5, colors = 'black', linewidths = 1, linestyles='dashed')
        
plt.scatter(X_ripley[indices_0_r], Y_ripley[indices_0_r], c='r', linewidths = 0.1, marker = 'o')
plt.scatter(X_ripley[indices_1_r], Y_ripley[indices_1_r], c='b', linewidths = 0.1, marker = 'v')
plt.scatter(X_relevance, Y_relevance, s=100, facecolors='none', edgecolors='b')

plt.show()

ValueError: operands could not be broadcast together with shapes (7,) (2,) 

In [26]:
# Verifies the result with the provided test sets 

if (set_option == 0):

    test_ripley = np.zeros((1000,3))

    # Converts the dataset from text file to Python arrays
    my_file = open('synth.te','r')

    index = 0

    for line in my_file:
        number = ''
        for i in range(len(line)):
            if (i == len(line)-2):
                test_ripley[index/3][index%3] = float(line[i])
                index += 1
            elif (line[i] == ' '):
                if (len(number) > 0):
                    test_ripley[index/3][index%3] = float(number)
                    index += 1
                number = ''
            else:
                number += line[i]

    my_file.close()

    
    # Counts the proportion of errors on the test set
    error_count = 0
    
    for i in range(1000):
        
        result = 0
        new_point = test_ripley[i,0:2]

        for j in range(len(alpha_indices)):

            index = alpha_indices[j]
            result += w[index]*Gaussian_kernel(data_points[index-1], new_point)
        
        if not (abs(logistic_sigmoid(result) - test_ripley[i,2]) < 0.5):
            
            error_count += 1

    print("The test error is " + str(error_count/10.) + "%.")
    
    

if (set_option == 1):

    data_test_pima = np.zeros((332,7))
    t_test_pima = np.zeros(332)

    # Converts the dataset from text file to Python arrays
    my_file = open('pima.te','r')

    index = 0

    for line in my_file:
        number = ''
        for i in range(len(line)):
            if (line[i] == ' '):
                if (len(number) > 0):
                    data_test_pima[index/7][index%7] = float(number)
                    index += 1
                number = ''
            elif (i == len(line) - 2):
                t_test_pima[index/7 - 1] = float(line[i])
            else:
                number += line[i]
                
    my_file.close()
    
    
    # Counts the number of errors on the test set
    error_count = 0
    
    for i in range(332):
        
        result = 0
        new_point = data_test_pima[i]

        for j in range(len(alpha_indices)):

            index = alpha_indices[j]
            result += w[index]*Gaussian_kernel(data_points[index-1], new_point)
        
        if not (abs(logistic_sigmoid(result) - t_test_pima[i]) < 0.5):
            
            error_count += 1

    print("The number of errors is " + str(error_count) + ".")
    print("The number of errors is " + str(error_count/3.32) + "%.")
    

The number of errors is 67.
The number of errors is 20.1807228916%.
